In [1]:
import os
import subprocess as sp
import numpy
import select
import sys

#set up ffmpeg
FFMPEG_BIN = "ffmpeg"
video = '556.mp4'
width = 1920
height = 1080
command = [ FFMPEG_BIN,
            '-i', video,
            '-f', 'image2pipe',
            '-pix_fmt', 'rgb24',
            '-vcodec', 'rawvideo', '-']
pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)

#return red value from each frame
def captureFrame():
    # read 1920*1080*3 bytes (= 1 frame)
    raw_image = pipe.stdout.read(width*height*3)
    # transform the byte read into a numpy array
    image =  numpy.fromstring(raw_image, dtype='uint8')
    frameTotalByteSize = width*height*3
    if len(image) < frameTotalByteSize:
        return False
    image = image.reshape((width,height,3))
    # throw away the data in the pipe's buffer.
    pipe.stdout.flush()
    totalRed = 0
    for col in range(0, width):
        for row in range(0, height):
            totalRed += image[col][row][0]
    return totalRed/1.0/width/height

print 'Loading the video'
rValueList = list()
while True:
    r = captureFrame()
    if not r:
        break
    else:
        rValueList.append(r)
    print r
print 'finished'

Loading the video
200.484855324
200.480170235
200.425559896
200.233194927
200.393378183
200.631685475
200.868954475
201.120486111
201.439062018
201.779889564
202.071319927
201.625255594
199.301302083
197.293290895
196.510572434
196.415004823
196.39206115
196.307606096
196.101388889
196.063744213
195.994953221
195.667468171
195.41380787
195.564964796
195.797244406
196.13488233
196.571943962
197.004016686
197.44536169
197.834257812
198.243578318
198.940271508
198.949349923
199.056457851
199.315832851
199.649478684
199.931431809
200.112841435
200.336533565
200.544332079
200.724728009
200.463932774
198.485649113
196.414710166
195.49971113
195.29527826
195.330274402
195.475193866
195.770810185
196.101536941
196.20005353
196.119071181
195.979987461
195.913237365
195.958085938
196.146508005
196.428148148
196.760552662
197.063800154
197.358118731
197.618797261
197.83501495
198.285418596
198.286467496
198.383234954
198.580099344
198.796251929
199.040847801
198.939323881
197.338429302
195.150531

In [2]:
#obtain rr intervals from red value
def constructRRListFromRedValueList(redValueList, fps):
    timePerFrame = 1.0/fps
    rrList = list()
    lastMaxIndex = False
    for i in range(1, redValueList-1):
        previousSlope = redValueList[i] - redValueList[i-1]
        afterSlope = redValueList[i+1] - redValueList[i]
        # if at that point it is concave down = local max
        if previousSlope*afterslope < 0 and previousSlope > 0:
            if lastMaxIndex:
                rrInterval = (i-lastMax)*timePerFrame
                rrList.append(rrInterval)
        lastMaxIndex = i
    return rrList

In [5]:
file = open('temp', 'w')
numpy.savetxt(file, rValueList)

In [3]:
print len(rValueList)


907
